In [11]:
# CS671_DEEP_LEARNING
# Programming Assignment 1
# Course Instructor : Aditya Nigam
# 04 March 2019
# Question2
# Submitted By Group_No_6

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

import numpy as np
import tensorflow as tf

In [12]:
# load data from external files
posi = np.load('q2_input/positions.npy')
mass = np.load('q2_input/masses.npy')
velo = np.load('q2_input/velocities.npy')

In [13]:
# create graph
graph = tf.get_default_graph()
with graph.as_default():
    thresh = tf.constant(.1, dtype=tf.float64, name='Treshold')
    G = tf.constant(667000.0, dtype=tf.float64, name='Gravitation_constant')
    dt = tf.constant(0.0001, dtype=tf.float64, name='Delta_t')
    m = tf.Variable(mass, dtype=tf.float64, name='Masses')
    v = tf.Variable(velo, dtype=tf.float64, name='Velocities')
    p = tf.Variable(posi, dtype=tf.float64, name='Positions')
    size_m = tf.size(m)
    rel = tf.cast(tf.Variable(tf.fill([size_m, size_m, 2],0.0)), dtype=tf.float64, name='Rel_pos')
    
    # manipuulations
    rel = tf.map_fn(lambda x: rel[x] + p - p[x], tf.range(size_m), dtype=tf.float64, name='Mapping_rel_pos')
    s_rel = tf.math.reduce_sum(rel*rel, 2, keepdims=True)
    mod_rel = tf.math.sqrt(s_rel, name='Mod_r')
    size = tf.size(mod_rel)-size_m
    flag = tf.cast(tf.reduce_sum(tf.where(mod_rel >= thresh, 1 * tf.ones_like(mod_rel), 0 * tf.ones_like(mod_rel))), dtype=tf.int32, name='Flag')
    cr_mod_rel = tf.math.reciprocal(tf.math.pow(mod_rel, 3))
    a = tf.math.multiply(G,m)
    a1 = tf.math.multiply(a, cr_mod_rel)
    a2 = tf.math.multiply(rel, a1, name='Acceleration_tensor')
    a3 = tf.where(tf.is_nan(a2), tf.zeros_like(a2), a2)
    a_net = tf.reduce_sum(a3, 1, name='Net_Acceleration')
    
    # output tensors
    v_out = tf.math.add(v, tf.math.multiply(a_net,dt), name="V_out")  
    p_out = tf.math.add(p, tf.math.add(tf.math.multiply(v, dt), 1/2*a_net*dt*dt), name='P_out')

# graph summary in logs to have a visual
# representation in Tensorboard.
summary = tf.summary.scalar('v', v)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [14]:
x = 0
vel = velo
pos = posi

# session loop
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    while(sess.run(flag, feed_dict={v:vel, p:pos}) == sess.run(size, feed_dict={v:vel, p:pos})):
        vel = sess.run(v_out, feed_dict={v:vel, p:pos})
        pos = sess.run(p_out, feed_dict={v:vel, p:pos})
        mod = sess.run(mod_rel, feed_dict={p:pos})
        x+=1

In [15]:
print("Total iterations: ", x)

# save output in file
np.save('q2_output/velocities.npy', vel)
np.save('q2_output/positions.npy', pos)

Total iterations:  58
